In [1]:
#
import os
import sys
import warnings
import dataclasses
from copy import deepcopy
from pathlib import Path
from typing import Dict, Union

#
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

# Stable Baseline3
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

#
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# from pyfolio import create_returns_tear_sheet
# from pyfolio import utils

#
from rl.plot.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from common.utils import now_time
from configuration.settings import ProjectDir, ExperimentDir
from rl.data.CompanyInfo import CompanyInfo
from rl.envs.StockTradingEnv import StockTradingEnv
from rl.experiments._1_same_bigger_data_fundamental.train import (
    Program,
    CustomDRLAgent,
    get_dataset,
    get_env_kwargs,
    dataset_name,
    base_cols,
    data_cols,
    ratios_cols,
    algorithm_name,
)

/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
def ignore_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)


ignore_warnings()

In [3]:
def main():
    program = Program(
        prj_dir=ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing")),
        exp_dir=ExperimentDir(Path(os.getcwd())),
        DEBUG=False,
    )
    program.dataset = get_dataset(
        pd.read_csv(program.exp_dir.out.datasets.joinpath(f"{dataset_name}.csv"), index_col=0), purpose="test"
    )
    program.exp_dir.check_and_create_dirs()
    return program


program = main()

In [4]:
print(base_cols)
print(data_cols)
print(ratios_cols)

['date', 'tic']
['open', 'high', 'low', 'close', 'volume']
['operatingProfitMargin', 'netProfitMargin', 'returnOnAssets', 'returnOnEquity', 'currentRatio', 'quickRatio', 'cashRatio', 'inventoryTurnover', 'receivablesTurnover', 'payablesTurnover', 'debtRatio', 'debtEquityRatio', 'priceEarningsRatio', 'priceBookValueRatio', 'dividendYield']


In [5]:
@dataclasses.dataclass
class LearnedAlgorithm:
    algorithm: str
    filename: Path
    learned_algorithm: Union[A2C, PPO, DDPG, A2C, TD3]
    df_account_value = pd.DataFrame()
    df_actions = pd.DataFrame()
    perf_stats_all = pd.DataFrame()


def get_algorithm(filename: Path):
    if "a2c" in filename.as_posix():
        return LearnedAlgorithm(algorithm="a2c", filename=filename, learned_algorithm=A2C.load(filename))


learned_algorithms = [get_algorithm(filepath) for filepath in program.exp_dir.out.algorithms.glob("*")]

In [6]:
# Test Learned Algorithms

print(len(learned_algorithms))

for algo in learned_algorithms:
    env_kwargs = get_env_kwargs(program.dataset)
    env_gym = StockTradingEnv(df=program.dataset, **env_kwargs)
    algo.df_account_value, algo.df_actions = CustomDRLAgent.DRL_prediction(
        model=algo.learned_algorithm, environment=env_gym
    )

24
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dimension: 29, State Space: 494
hit end!
Stock Dime

In [7]:
# print(learned_algorithms[0].df_actions)
print(learned_algorithms[0].df_account_value)

           date  account_value
0    2019-04-12   1.000000e+06
1    2019-04-15   9.999540e+05
2    2019-04-16   9.999689e+05
3    2019-04-17   9.968838e+05
4    2019-04-18   9.995996e+05
..          ...            ...
923  2022-12-09   1.304557e+06
924  2022-12-12   1.326564e+06
925  2022-12-13   1.334561e+06
926  2022-12-14   1.332686e+06
927  2022-12-15   1.301198e+06

[928 rows x 2 columns]


In [8]:
best_stats: LearnedAlgorithm = None
worst_stats: LearnedAlgorithm = None

metric = "Annual return"

for algo in learned_algorithms:
    perf_stats_all = backtest_stats(account_value=algo.df_account_value)
    perf_stats_all = pd.DataFrame(perf_stats_all)

    if best_stats is None or worst_stats is None:
        algo.perf_stats_all = perf_stats_all
        best_stats = algo
        worst_stats = algo

    if perf_stats_all.loc[metric][0] > best_stats.perf_stats_all.loc[metric][0]:
        best_stats = algo
    elif perf_stats_all.loc[metric][0] < worst_stats.perf_stats_all.loc[metric][0]:
        worst_stats = algo

KeyError: 'Annual return'

In [27]:
start = program.dataset["date"].min()
end = program.dataset["date"].max()
print(f"Start: {start}  -  End: {end}")

Start: 2019-04-12  -  End: 2022-12-15


In [28]:
# Baseline
baseline_df = get_baseline("^DJI", start, end)

In [29]:
baseline_df

,date,open,high,low,close,volume,dividends,stock splits
Date,,,,,,,,
2019-04-12,2019-04-12,26357.789062,26436.679688,26309.720703,26412.300781,369910000,0,0
2019-04-15,2019-04-15,26407.759766,26424.849609,26316.419922,26384.769531,250620000,0,0
2019-04-16,2019-04-16,26482.189453,26530.710938,26397.189453,26452.660156,302340000,0,0
2019-04-17,2019-04-17,26468.529297,26501.019531,26391.859375,26449.539062,367780000,0,0
2019-04-18,2019-04-18,26463.369141,26602.419922,26444.529297,26559.539062,332850000,0,0
...,...,...,...,...,...,...,...,...
2022-12-08,2022-12-08,33695.968750,33899.171875,33642.460938,33781.480469,271830000,0,0
2022-12-09,2022-12-09,33746.710938,33844.628906,33462.648438,33476.460938,287630000,0,0
2022-12-12,2022-12-12,33519.500000,34005.039062,33519.500000,34005.039062,313440000,0,0


In [30]:
print(baseline_df.index.min())
print(baseline_df.index.max())

2019-04-12
2022-12-14


In [31]:
stats = backtest_stats(baseline_df, value_col_name="close")

Annual return          0.070772
Cumulative returns     0.286005
Annual volatility      0.232728
Sharpe ratio           0.411239
Calmar ratio           0.190831
Stability              0.561379
Max drawdown          -0.370862
Omega ratio            1.087791
Sortino ratio          0.567932
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.953450
Daily value at risk   -0.028941
dtype: float64


ModuleNotFoundError: No module named 'rl'

In [33]:
# account_value = df_account_value
# baseline_ticker = "^DJI"
# baseline_start = program.dataset.index.min()
# baseline_end = program.dataset.index.max()
# value_col_name = "account_value"
#
# df = deepcopy(account_value)
# df["date"] = pd.to_datetime(df["date"])
# test_returns = get_daily_return(df, value_col_name=value_col_name)
#
# baseline_df = get_baseline(ticker=baseline_ticker, start=baseline_start, end=baseline_end)
#
# baseline_df["date"] = pd.to_datetime(baseline_df["date"], format="%Y-%m-%d")
# baseline_df = pd.merge(df[["date"]], baseline_df, how="left", on="date")
# baseline_df = baseline_df.fillna(method="ffill").fillna(method="bfill")
# baseline_returns = get_daily_return(baseline_df, value_col_name="close")
#
# with pyfolio.plotting.plotting_context(font_scale=1.1):
#     pyfolio.create_full_tear_sheet(
#         returns=test_returns,
#         benchmark_rets=baseline_returns,
#         set_context=False,
#     )
#

Start date,2019-04-12
End date,2022-12-15
Total months,44
,Backtest
Annual return,5.772%
Cumulative returns,22.954%
Annual volatility,29.539%
Sharpe ratio,0.34
Calmar ratio,0.13
Stability,0.36
Max drawdown,-44.159%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,44.16,2020-02-12,2020-03-20,2021-01-06,236
1,32.13,2021-11-02,2022-09-27,NaT,NaN
2,9.42,2019-07-23,2019-08-14,2019-09-11,37
3,7.11,2019-04-30,2019-06-03,2019-06-20,38
4,6.81,2021-08-27,2021-09-21,2021-10-18,37


Stress Events,mean,min,max
New Normal,0.04%,-15.71%,13.63%


In [34]:
#
# df = deepcopy(account_value)
# df["date"] = pd.to_datetime(df["date"])
# test_returns = get_daily_return(df, value_col_name=value_col_name)
#
# positions = utils.check_intraday("infer", test_returns, None, None)
#
# create_returns_tear_sheet(
#     test_returns,
#     positions=positions,
#     transactions=None,
#     live_start_date=None,
#     cone_std=(1.0, 1.5, 2.0),
#     benchmark_rets=None,
#     bootstrap=False,
#     turnover_denom="AGB",
#     header_rows=None,
#     set_context=True
# )

Start date,2019-04-12
End date,2022-12-15
Total months,44
,Backtest
Annual return,5.772%
Cumulative returns,22.954%
Annual volatility,29.539%
Sharpe ratio,0.34
Calmar ratio,0.13
Stability,0.36
Max drawdown,-44.159%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,44.16,2020-02-12,2020-03-20,2021-01-06,236
1,32.13,2021-11-02,2022-09-27,NaT,NaN
2,9.42,2019-07-23,2019-08-14,2019-09-11,37
3,7.11,2019-04-30,2019-06-03,2019-06-20,38
4,6.81,2021-08-27,2021-09-21,2021-10-18,37
